In [1]:
import os

os.environ['CUDA_VISIBLE_DEVICES'] = '4,5,6,7'

In [2]:
from multigpus_repl import init_multigpus_repl, multigpus

/root/.venv/lib/python3.12/site-packages/torch/cuda/__init__.py:63: FutureWarning: The pynvml package is deprecated. Please install nvidia-ml-py instead. If you did not install pynvml directly, please report this to the maintainers of the package that installed pynvml for you.
  import pynvml  # type: ignore[import]


In [3]:
init_multigpus_repl()

[GPU 3] Worker 3/4 initialized on GPU 3 on localhost:12355[GPU 0] Worker 0/4 initialized on GPU 0 on localhost:12355[GPU 1] Worker 1/4 initialized on GPU 1 on localhost:12355[GPU 2] Worker 2/4 initialized on GPU 2 on localhost:12355

In [27]:
%%multigpus

import math
import torch
import torch.nn.init as init
import torch.nn as nn
from torch.distributed._tensor import DTensor, Shard, Replicate, distribute_tensor
from torch.distributed.device_mesh import init_device_mesh

device_type = torch.accelerator.current_accelerator()
device_mesh = init_device_mesh(device_type.type, (1, 4), mesh_dim_names=("dp", "tp"))
tp_mesh = device_mesh["tp"]
dp_mesh = device_mesh["dp"]
dp_rank = dp_mesh.get_local_rank()
dp_world_size = dp_mesh.size()
        
gate_up_proj = nn.Parameter(torch.zeros(32, 100, 100, dtype=torch.bfloat16))
with torch.no_grad():
    init.kaiming_uniform_(gate_up_proj, a=math.sqrt(5))

gate_up_proj = distribute_tensor(
    gate_up_proj,
    device_mesh=tp_mesh,
    placements=[Shard(0)]
)

x = torch.randn((100, 100), dtype=torch.bfloat16).cuda()
print(x @ gate_up_proj[0].to_local())

[GPU 0] tensor([[ 0.0786,  0.1060, -0.0454,  ..., -0.0708, -0.0520, -0.0187],
        [-0.0532,  0.0449,  0.0089,  ..., -0.0059,  0.0630,  0.0566],
        [ 0.0055,  0.0708,  0.0435,  ..., -0.1187, -0.0067, -0.0145],
        ...,
        [-0.0240, -0.0571, -0.0884,  ..., -0.0018, -0.0481,  0.0361],
        [-0.1484,  0.0732, -0.0146,  ...,  0.0073,  0.0371, -0.0693],
        [ 0.0074,  0.0024, -0.0918,  ...,  0.0027, -0.0913, -0.0452]],
       device='cuda:0', dtype=torch.bfloat16, grad_fn=<MmBackward0>)
[GPU 1] tensor([[ 0.0786,  0.1060, -0.0454,  ..., -0.0708, -0.0520, -0.0187],
        [-0.0532,  0.0449,  0.0089,  ..., -0.0059,  0.0630,  0.0566],
        [ 0.0055,  0.0708,  0.0435,  ..., -0.1187, -0.0067, -0.0145],
        ...,
        [-0.0240, -0.0571, -0.0884,  ..., -0.0018, -0.0481,  0.0361],
        [-0.1484,  0.0732, -0.0146,  ...,  0.0073,  0.0371, -0.0693],
        [ 0.0074,  0.0024, -0.0918,  ...,  0.0027, -0.0913, -0.0452]],
       device='cuda:1', dtype=torch.bfloat16, g

In [29]:
%%multigpus

x = torch.randn((100, 100), dtype=torch.bfloat16).cuda()
print(x @ gate_up_proj[10].to_local())

[GPU 0] tensor([[ 0.0649, -0.0811, -0.0947,  ..., -0.0024, -0.0991,  0.0464],
        [ 0.1089,  0.0630,  0.0024,  ...,  0.0400,  0.0012, -0.0713],
        [ 0.0201, -0.0024, -0.0311,  ...,  0.0398,  0.1221, -0.0347],
        ...,
        [-0.0275,  0.1064, -0.0430,  ...,  0.1094, -0.0640, -0.1260],
        [ 0.0894, -0.1050, -0.1196,  ..., -0.0184,  0.0459,  0.0236],
        [-0.0080, -0.0400, -0.0718,  ..., -0.0352, -0.0986, -0.0850]],
       device='cuda:0', dtype=torch.bfloat16, grad_fn=<MmBackward0>)
[GPU 1] tensor([[ 0.0649, -0.0811, -0.0947,  ..., -0.0024, -0.0991,  0.0464],
        [ 0.1089,  0.0630,  0.0024,  ...,  0.0400,  0.0012, -0.0713],
        [ 0.0201, -0.0024, -0.0311,  ...,  0.0398,  0.1221, -0.0347],
        ...,
        [-0.0275,  0.1064, -0.0430,  ...,  0.1094, -0.0640, -0.1260],
        [ 0.0894, -0.1050, -0.1196,  ..., -0.0184,  0.0459,  0.0236],
        [-0.0080, -0.0400, -0.0718,  ..., -0.0352, -0.0986, -0.0850]],
       device='cuda:1', dtype=torch.bfloat16, g